In [1]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision import models
from torchvision import datasets
from torchvision import transforms

from PIL import Image
import os
import numpy as np
import pandas as pd

In [2]:
device = (
    torch.device('cuda') if torch.cuda.is_available()
    else torch.device('cpu'))

In [ ]:
def train(model,
         n_epochs,
         criterion,
         optimizer,
         train_loader,
         val_loader):
    
    history = []

    for epoch in range(1, n_epochs+1):
        
        train_loss = 0
        valid_loss = 0

        train_acc = 0
        valid_acc = 0

        model.train()

        for data, label in train_loader:
            
            data, label = data.to(device=device), label.to(device=device)
            output = model(data)

            loss = criterion(output, label)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * data.size(0)

            _, prediction = torch.max(output, dim=1)
            correct_tensor = prediction.eq(label.data.view_as(prediction))
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
            train_acc += accuracy.item() * data.size(0)

        model.eval()        

        with torch.no_grad():

            for data, label in val_loader:
                data, label = data.to(device=device), label.to(device=device)

                output = model(data)
                loss = criterion(output, label)
                valid_loss += loss.item() * data.size(0)

                _, prediction = torch.max(output, dim=1)
                correct_tensor = prediction.eq(label.data.view_as(prediction))
                accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
                valid_acc += accuracy.item() * data.size(0)

            train_loss = train_loss / len(train_loader.dataset)
            valid_loss = valid_loss / len(val_loader.dataset)

            train_acc = train_acc / len(train_loader.dataset)
            valid_acc = valid_acc / len(val_loader.dataset)

            history.append([train_loss, valid_loss, train_acc, valid_acc])

            if epoch % 2 == 0:
                print(f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {valid_loss:.4f}')
                print(f'\t\tTraining Accuracy: {100 * train_acc:.2f}%\t Validation Accuracy: {100 * valid_acc:.2f}%')

    history = pd.DataFrame(history, columns=['train_loss', 'valid_loss', 'train_acc', 'valid_acc'])

    return history

In [ ]:
def test(model, test_loader, criterion):
  
    model.to(device=device)
    
    classes = []
    acc_results = np.zeros(len(test_loader.dataset))
    i = 0

    model.eval()
    with torch.no_grad():
        for data, labels in test_loader:
            data, labels = data.to(device=device), labels.to(device=device)
            output = model(data)

            for prediction, true in zip(output, labels):
                _, prediction = prediction.unsqueeze(0).topk(1)
                correct = prediction.eq(true.unsqueeze(0))
                acc_results[i] = correct.cpu()
                classes.append(model.idx_to_class[true.item()][10:])
                i+=1

    results = pd.DataFrame({
      'class': classes,
      'results': acc_results    
    })
    results = results.groupby(classes).mean()
    return results

In [ ]:
image_transforms = {
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=315, scale=(0.95, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test':
    transforms.Compose([
        transforms.Resize(size=299),
        transforms.CenterCrop(size=299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [ ]:
data_dir = './dataset/stanford-dogs-dataset/images/Images/'

all_data = datasets.ImageFolder(root=data_dir)

train_data_len = int(len(all_data)*0.8)
valid_data_len = int((len(all_data) - train_data_len)/2)
test_data_len = int(len(all_data) - train_data_len - valid_data_len)

train_data, val_data, test_data = random_split(all_data, [train_data_len, valid_data_len, test_data_len])

train_data.dataset.transform = image_transforms['train']
val_data.dataset.transform = image_transforms['test']
test_data.dataset.transform = image_transforms['test']

batch_size = 128

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle = True)
val_loader = DataLoader(val_data, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test_data, batch_size = batch_size, shuffle = False)

In [3]:
net = models.resnet18(pretrained=True)

for param in net.parameters():
    param.requires_grad = False
    
num_in = net.fc.in_features
num_out = 120
net.fc = nn.Sequential(
    nn.Linear(
        in_features = num_in,
        out_features = num_out,
        bias=True),
    nn.LogSoftmax(dim=1))

net.class_to_idx = all_data.class_to_idx
net.idx_to_class = {
    idx: class_
    for class_, idx in net.class_to_idx.items()
}

net.to(device=device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
learning_rate = 0.0005

criterion = nn.NLLLoss()
optimizer = optim.Adam(net.parameters(), lr = learning_rate)

In [15]:
history = train(
    net,
    n_epochs=30,
    criterion = criterion,
    optimizer = optimizer,
    train_loader = train_loader,
    val_loader = val_loader)

torch.save(net.state_dict(), './dog_prediction.pt')

In [9]:
print(test(net, test_loader, criterion))

In [10]:
print(history)